Importamos Tigramite y preparamos los datos

In [ ]:
# backend/notebooks/causal_analysis.ipynb - Celda corregida
import pandas as pd
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
from sklearn.preprocessing import LabelEncoder
import joblib

# Cargar los datos
df = pd.read_csv('../data/unified_houses_madrid.csv')

# Expandir las variables para análisis causal MÁS COMPLETO
causal_columns = [
    # === FÍSICAS BÁSICAS ===
    'sq_mt_built', 'sq_mt_useful', 'n_rooms', 'n_bathrooms', 'floor',
    'n_floors', 'sq_mt_allotment',
    
    # === UBICACIÓN (CLAVE para causalidad) ===
    'latitude', 'longitude', 'district', 'neighborhood',
    
    # === TEMPORAL ===
    'built_year',
    
    # === CATEGÓRICAS IMPORTANTES ===  
    'house_type', 'energy_certificate',
    
    # === BOOLEANAS ESTRUCTURALES ===
    'has_lift', 'is_exterior', 'has_parking', 'is_new_development',
    'is_renewal_needed',
    
    # === COMODIDADES COMPLETAS ===
    'has_central_heating', 'has_individual_heating', 'has_ac',
    'has_garden', 'has_pool', 'has_terrace', 'has_storage_room',
    'is_furnished',
    
    # === ORIENTACIÓN COMPLETA ===
    'is_orientation_north', 'is_orientation_south',
    'is_orientation_east', 'is_orientation_west',
    
    # === OBJETIVOS Y PRECIOS ===
    'buy_price', 'rent_price', 'buy_price_by_area'
]

print(f"Total variables para análisis causal: {len(causal_columns)}")

# Filtrar DataFrame
causal_df = df[causal_columns].copy()

print("=== ANTES DEL PREPROCESAMIENTO ===")
print(f"Shape: {causal_df.shape}")
print(f"Tipos de datos:")
print(causal_df.dtypes)
print(f"Valores nulos:")
print(causal_df.isnull().sum())

# === PREPROCESAMIENTO CORRECTO ===

# 1. Separar variables por tipo
numeric_columns = causal_df.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = causal_df.select_dtypes(include=['object', 'category']).columns
boolean_columns = causal_df.select_dtypes(include=['bool']).columns

print(f"\n=== ANÁLISIS DE COLUMNAS ===")
print(f"Numéricas: {list(numeric_columns)}")
print(f"Categóricas: {list(categorical_columns)}")
print(f"Booleanas: {list(boolean_columns)}")

# 2. Convertir booleanas a numéricas (0/1)
for col in boolean_columns:
    causal_df[col] = causal_df[col].astype(int)

# 3. Manejar variables categóricas
for col in categorical_columns:
    if causal_df[col].dtype == 'object':
        # Opción A: Label Encoding (mejor para PCMCI)
        le = LabelEncoder()
        causal_df[col] = le.fit_transform(causal_df[col].fillna('Unknown'))
        print(f"Encoded {col}: {le.classes_}")
        
        # Opción B: One-Hot Encoding (comentado, usa solo si necesario)
        # dummy_df = pd.get_dummies(causal_df[col], prefix=col, drop_first=True)
        # causal_df = pd.concat([causal_df.drop(col, axis=1), dummy_df], axis=1)

# 4. Rellenar valores faltantes DESPUÉS de codificación
causal_df = causal_df.fillna(causal_df.median(numeric_only=True))

# 5. Asegurar que todas las columnas son numéricas
causal_df = causal_df.select_dtypes(include=['float64', 'int64'])

print(f"\n=== DESPUÉS DEL PREPROCESAMIENTO ===")
print(f"Shape: {causal_df.shape}")
print(f"Columnas finales: {list(causal_df.columns)}")
print(f"Todos los tipos son numéricos: {all(causal_df.dtypes.apply(lambda x: x in ['float64', 'int64']))}")

# Convertir a array numpy
data_array = causal_df.values

# Crear DataFrame de Tigramite
dataframe = pp.DataFrame(data_array, var_names=list(causal_df.columns))

# Configurar PCMCI
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr)

# Ejecutar PCMCI
print("\n=== EJECUTANDO PCMCI ===")
results = pcmci.run_pcmci(tau_max=3, pc_alpha=0.05)  # Reducido tau_max por más variables
results['var_names'] = list(causal_df.columns)

# Guardar resultados
joblib.dump(results, '../data/models/pcmci_results.joblib')
joblib.dump(causal_df.columns.tolist(), '../data/models/pcmci_var_names.joblib')

print("✅ Análisis completado y guardado")

=== ANTES DEL PREPROCESAMIENTO ===
Shape: (6735, 14)
Tipos de datos:
sq_mt_built           float64
n_rooms                 int64
n_bathrooms             int64
n_floors                int64
sq_mt_allotment       float64
floor                   int64
is_renewal_needed        bool
has_lift                 bool
is_exterior              bool
energy_certificate      int64
has_parking              bool
sq_mt_useful          float64
rent_price              int64
buy_price               int64
dtype: object
Valores nulos:
sq_mt_built              0
n_rooms                  0
n_bathrooms              0
n_floors                 0
sq_mt_allotment          0
floor                    0
is_renewal_needed        0
has_lift                 0
is_exterior              0
energy_certificate       0
has_parking              0
sq_mt_useful          3658
rent_price               0
buy_price                0
dtype: int64

=== ANÁLISIS DE COLUMNAS ===
Numéricas: ['sq_mt_built', 'n_rooms', 'n_bathrooms', 'n_floor

In [2]:
import joblib
import os

# Función para guardar el modelo PCMCI y el DataFrame de Tigramite
def guardar_modelo_pcmci(modelo, dataframe, ruta_modelo='../data/models/pcmci_model.joblib', ruta_dataframe='../data/models/pcmci_dataframe.joblib'):
    """
    Guarda el modelo PCMCI y el DataFrame de Tigramite.
    
    Args:
        modelo: El modelo PCMCI entrenado
        dataframe: El DataFrame de Tigramite
        ruta_modelo (str): Ruta donde guardar el modelo
        ruta_dataframe (str): Ruta donde guardar el DataFrame
    """
    # Crear la carpeta 'models' si no existe
    os.makedirs(os.path.dirname(ruta_modelo), exist_ok=True)
    
    # Guardar el modelo y el DataFrame
    joblib.dump(modelo, ruta_modelo)
    joblib.dump(dataframe, ruta_dataframe)
    
    print(f"Modelo PCMCI guardado en: {ruta_modelo}")
    print(f"DataFrame de Tigramite guardado en: {ruta_dataframe}")

# Guardar el modelo PCMCI y el DataFrame de Tigramite
guardar_modelo_pcmci(pcmci, dataframe)


Modelo PCMCI guardado en: ../data/models/pcmci_model.joblib
DataFrame de Tigramite guardado en: ../data/models/pcmci_dataframe.joblib


Explicación del código
Importar bibliotecas necesarias: Importamos las bibliotecas necesarias para el análisis.
Cargar los datos: Cargamos los datos desde un archivo CSV.
Seleccionar columnas relevantes: Seleccionamos las columnas relevantes para el análisis de causalidad.
Filtrar el DataFrame: Filtramos el DataFrame para incluir solo las columnas relevantes y rellenamos los valores faltantes.
Convertir DataFrame a un array numpy: Convertimos el DataFrame a un array numpy.
Crear un objeto DataFrame de Tigramite: Creamos un objeto DataFrame de Tigramite.
Configurar el test de independencia usando ParCorr: Configuramos el test de independencia usando ParCorr.
Configurar PCMCI: Configuramos PCMCI.
Ejecutar PCMCI para encontrar relaciones causales: Ejecutamos PCMCI para encontrar relaciones causales.
Inspeccionar las claves disponibles en los resultados: Imprimimos las claves disponibles en los resultados.
Documentar las relaciones causales encontradas: Iteramos sobre la matriz graph y documentamos las relaciones causales encontradas de manera legible, verificando si contienen '-->' o 'o-o'.

 En este análisis se utiliza el algoritmo PCMCI (Peter and Clark Momentary Conditional Independence) para identificar relaciones causales entre diferentes variables de un conjunto de datos de propiedades inmobiliarias. A continuación, se detallan los pasos realizados:

1. **Carga de datos**:
- Se cargan datos desde un archivo CSV que contiene información sobre propiedades inmobiliarias en Madrid.
- Se seleccionan columnas relevantes relacionadas con características de las propiedades y sus precios.
 
2. **Preprocesamiento**:
- Solo se consideran las columnas numéricas para el análisis.
- Los valores faltantes se rellenan con la media de cada columna para garantizar un conjunto de datos completo.

3. **Configuración del análisis causal**:
- Se convierte el DataFrame en un formato compatible con Tigramite (`pp.DataFrame`).
- Se utiliza el test de independencia condicional basado en correlación parcial (`ParCorr`).
- Se ejecuta PCMCI con un máximo de 5 retardos temporales (`tau_max=5`) para identificar relaciones causales.

4. **Resultados**:
- Los resultados incluyen un grafo (`graph`) que indica las relaciones causales detectadas entre las variables.
- Cada relación puede ser:
- **`var1 --> var2`**: `var1` tiene un efecto causal directo sobre `var2`.
- **`var1 o-o var2`**: Existe una relación no orientada entre `var1` y `var2` (no se puede determinar la dirección).
- **`var1 --- var2`**: Una relación detectada que no necesariamente es causal.

5. **Interpretación de ejemplos de relaciones causales**:
- Si el grafo contiene relaciones como `sq_mt_built --> rent_price`, esto indica que la superficie construida tiene un efecto directo sobre el precio de renta.
- Una relación como `n_rooms --> buy_price (tau=1)` sugiere que el número de habitaciones afecta al precio de compra con un retardo temporal.
- Relaciones no orientadas (`o-o`) pueden indicar que existe una correlación entre las variables, pero no se puede determinar la dirección causal debido a posibles factores externos no observados.


In [3]:
import matplotlib.pyplot as plt

causal_relationships = []
# Crear gráficos para visualizar las relaciones causales
for var1, relation, var2 in causal_relationships:
    plt.figure(figsize=(10, 6))
    plt.scatter(causal_df[var1], causal_df[var2], alpha=0.5)
    plt.title(f'Relación causal: {var1} {relation} {var2}')
    plt.xlabel(var1)
    plt.ylabel(var2)
    plt.grid(True)
    plt.show()

In [4]:
import folium

# Crear un mapa centrado en Madrid
m = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Añadir marcadores para cada propiedad
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Precio de compra: {row['buy_price']}, Precio de renta: {row['rent_price']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Guardar el mapa en un archivo HTML
m.save('mapa_distribucion_geografica.html')